In [1]:
cd ..

/Users/ulfaslak/Dropbox/school/sem12/MScProject/Academic/code/pipeline_code


### Simulate structured data

In [2]:
from pareto_clustering.dependencies.matlab import generate_poisson_dataset
X_sim, labels_sim = generate_poisson_dataset.generate_poisson_dataset(4,100,100,0.0)

### Compute T-ratio matrix

In [8]:
from pareto_clustering.cluster.build_S import Build_S
_, T, _ = Build_S(X_sim,5,sample_size=1.0, remove_outliers=False).main(visualize=False)

0% 50/4950
10% 545/4950
20% 1040/4950
30% 1535/4950
40% 2030/4950
50% 2525/4950
60% 3020/4950
70% 3515/4950
80% 4010/4950
90% 4505/4950
100% 4950/4950


### Infer communities

In [11]:
from pareto_clustering.cluster import cluster_Infomap
pred_communities = cluster_Infomap.community_detection_Infomap(T)

### Compare inferred communities to true communities

In [13]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

def flatten(l):
    return [item for sublist in l for item in sublist]

pred_communities_sorted = sorted(pred_communities.values(), key=lambda x: (x[0],x[-1]))

true_stringyfied = flatten([[str(i)]*v for i,v in enumerate(labels_sim)])
pred_stringyfied = flatten([[str(i)]*len(v) for i,v in enumerate(pred_communities_sorted)])

edit_distance = levenshtein(true_stringyfied, pred_stringyfied)

print "Edit distance:", edit_distance
print "Accuracy:", 100-edit_distance*0.5/len(pred_stringyfied)*100

Edit distance: 2
Accuracy: 99.0


/Users/ulfaslak/miniconda/lib/python2.7/site-packages/ipykernel/__main__.py:26: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
